In [2]:
!pip install hazm

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 14.8 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9300 sha256=1d8e101d546d7dbe84b08b3ea513d8c013a70c0722c89fe85182d449878d3ed0
  Stored in directory: /root/.cache/pip/wheels/49/20/47/f03dfa8a7239c54cbc44ff7389eefbf888d2c1873edaaec888
Successfully built flashtext
  Attempting uninstall: numpy
  

In [31]:
import json
import numpy as np
from hazm import Normalizer, stopwords_list
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [29]:
with open("aggregate.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("Test_data.json", "r", encoding="utf-8") as f:
    eval_data = json.load(f)

In [35]:
normalizer = Normalizer()
stop_words = stopwords_list()

def convert_persian_digits(text):
    persian_digits = '۰۱۲۳۴۵۶۷۸۹'
    english_digits = '0123456789'
    trans_table = str.maketrans(''.join(persian_digits), ''.join(english_digits))
    return text.translate(trans_table)

def preprocess(text):
    text = normalizer.normalize(text)
    text = convert_persian_digits(text)
    return text

In [33]:
contexts = [preprocess(item['context']) for item in data]

vectorizer = TfidfVectorizer(stop_words=stop_words)
context_vectors = vectorizer.fit_transform(contexts)

results = []

for eval_item in eval_data:
    q_id = eval_item['id']
    question = preprocess(eval_item['question'])

    q_vec = vectorizer.transform([question])
    similarities = cosine_similarity(q_vec, context_vectors).flatten()
    top_indices = similarities.argsort()[::-1][:3]

    top_results = []
    for idx in top_indices:
        top_results.append({
            "context": data[idx]["context"],
            "similarity": float(similarities[idx])
        })

    results.append({
        "id": q_id,
        "question": eval_item["question"],
        "answer": eval_item["answer"],
        "retrieved": top_results
    })

In [34]:
with open("tfidf_eval_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)